In [ ]:
import pandas as pd
hits = pd.read_csv("data/final_hits_10.csv")

#hits.head(5)
#hits.info()
#hits.describe()

In [ ]:
import pandas as pd
products = pd.read_csv("data/final_products_10.csv")

# products.head(5)
# products.info()

In [ ]:
import pandas as pd

sessions = pd.read_csv("data/final_sessions_10.csv")

# sessions.head(5)
# sessions.info()

# Data Import & Cleaning

### Converting to the standard currency in USA

Перевіряємо максимум перед діленям

In [ ]:
print('Max of transactionRevenue:', hits["transactionRevenue"].max())
print('Max of transactionTax:', hits["transactionTax"].max())
print('Max of transactionShipping:', hits["transactionShipping"].max())
print('Max of productRevenue:', products["productRevenue"].max())
print('Max of productPrice:', products["productPrice"].max())

In [ ]:
hits["transactionRevenue_converted"] = hits["transactionRevenue"] / 1000000
hits["transactionTax_converted"] = hits["transactionTax"] / 1000000
hits["transactionShipping_converted"] = hits["transactionShipping"] / 1000000
products["productRevenue_converted"] = products["productRevenue"] / 1000000
products["productPrice_converted"] = products["productPrice"] / 1000000

Тепер перевіряємо, чи коректно все поділилося

In [ ]:
print('Max of transactionRevenue:', hits["transactionRevenue_converted"].max())
print('Max of transactionTax:', hits["transactionTax_converted"].max())
print('Max of transactionShipping:', hits["transactionShipping_converted"].max())
print('Max of productRevenue:', products["productRevenue_converted"].max())
print('Max of productPrice:', products["productPrice_converted"].max())

In [ ]:
import numpy as np

placeholders = ['(not set)', '', '(none)', '(not provided)', 'not available in demo dataset']

def standardize_missing_values(df):
    return df.replace(placeholders, np.nan)

hits = standardize_missing_values(hits)
products = standardize_missing_values(products)
sessions = standardize_missing_values(sessions)

print("Missing values in final_hits_10:\n", hits.isnull().sum())
print("Missing values in final_products_10:\n", products.isnull().sum())
print("Missing values in final_sessions_10:\n", sessions.isnull().sum())

### Handle Geographic Data Completeness Variations

In [ ]:
print(sessions[['continent', 'subcontinent', 'country', 'city']].isnull().sum())
print()
sessions['clean_continent'] = sessions['continent'].fillna('No data')
sessions['clean_subcontinent'] = sessions['subcontinent'].fillna('No data')
sessions['clean_country'] = sessions['country'].fillna('No data')
sessions['clean_city'] = sessions['city'].fillna('No data')
print("Перевіряємо, чи все коректно замінилося")
print(sessions[['clean_continent', 'clean_subcontinent', 'clean_country', 'clean_city']].isnull().sum())

### Proper Date/Time Formatting and Timezone Considerations

In [ ]:
hits['converted_visitStartTime'] = pd.to_datetime(hits['visitStartTime'], unit='s', utc=True)

print("Sample of converted visitStartTime:", hits['converted_visitStartTime'].head())

In [ ]:
sessions['converted_date'] = pd.to_datetime(sessions['date'])

print("Sample of converted date:", sessions['converted_date'].head())

In [ ]:
hits['derived_hour'] = hits['converted_visitStartTime'].dt.hour
hits['derived_minute'] = hits['converted_visitStartTime'].dt.minute
print("Hour mismatches:", (hits['hour'] != hits['derived_hour']).sum())
print("Minute mismatches:", (hits['minute'] != hits['derived_minute']).sum())

# Drop якщо немає помилок
#hits = hits.drop(['derived_hour', 'derived_minute'], axis=1)

### Browser/OS categorization

In [ ]:
print("Unique browsers:", sessions['browser'].unique())
print()
print("Unique OS:", sessions['operatingSystem'].unique())

In [ ]:
top_browsers = sessions["browser"].value_counts().nlargest(5).index
sessions["grouped_browser"] = sessions["browser"].apply(lambda x: x if x in top_browsers else "Other")

top_os = sessions["operatingSystem"].value_counts().nlargest(5).index
sessions["grouped_operatingSystem"] = sessions["operatingSystem"].apply(lambda x: x if x in top_os else "Other")

In [ ]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

# Derived Column Creation

## Time-Based Calculations:
### Precise hit timestamps

In [ ]:
hits['hitTimestamp'] = hits['converted_visitStartTime'] + pd.to_timedelta(hits['time'], unit='ms')
print(hits["hitTimestamp"].head(5))

### Session duration

In [ ]:
session_durations = hits.groupby('uniqueVisitId')['hitTimestamp'].agg(['min', 'max']).reset_index()
session_durations['sessionDurationSec'] = (session_durations['max'] - session_durations['min']).dt.total_seconds()

sessions = sessions.merge(
    session_durations[['uniqueVisitId', 'sessionDurationSec']],
    on='uniqueVisitId',
    how='left'
)

### Time-of-day and day-of-week analysis

In [ ]:
hits['time_of_day'] = hits['hitTimestamp'].dt.hour
hits['day_of_week'] = hits['hitTimestamp'].dt.day_name()

sessions['day_of_week'] = sessions['converted_date'].dt.day_name()

In [ ]:
def categorize_time_of_day(hour):
    if 0 <= hour < 6:
        return 'Night'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'
hits['time_of_day_category'] = hits['time_of_day'].apply(categorize_time_of_day)

In [ ]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

## Content & Behavior Analysis:
### Page hierarchy extraction

In [ ]:
def split_page_path(path):
    if pd.isna(path):
        return [None] * 4
    
    levels = [level for level in path.strip('/').split('/') if level]
    levels += [None] * (4 - len(levels)) if len(levels) < 4 else []
    return pd.Series(levels[:4], index=['pageLevel1', 'pageLevel2', 'pageLevel3', 'pageLevel4'])

page_hierarchy = hits['pagePath'].apply(split_page_path)
hits = pd.concat([hits, page_hierarchy], axis=1)

###  Landing / Exit pages

In [ ]:
hits['isLandingPage'] = hits['isEntrance'].astype(str).str.lower().isin(['true', '1'])
hits['isExitPage'] = hits['isExit'].astype(str).str.lower().isin(['true', '1'])

# Визначення першої сторінки сесії
landing_pages = hits[hits['isLandingPage']].groupby('uniqueVisitId')['pagePath'].first().reset_index()
landing_pages.columns = ['uniqueVisitId', 'landingPage']

# Визначення останньої сторінки сесії
exit_pages = hits[hits['isExitPage']].groupby('uniqueVisitId')['pagePath'].last().reset_index()
exit_pages.columns = ['uniqueVisitId', 'exitPage']  # <- Ось тут виправлено

hits = hits.merge(landing_pages, on='uniqueVisitId', how='left').merge(exit_pages, on='uniqueVisitId', how='left')

In [ ]:
print(hits.head(5))

### New vs. Returning

In [ ]:
sessions['visitorType'] = sessions['visitnumber'].apply(lambda x: 'New' if x == 1 else 'Returning')

In [ ]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

## Marketing & Geographic Insights:
### Marketing channel hierarchies and groupings

In [ ]:
sessions['sourceMedium'] = sessions['source'] + ' / ' + sessions['medium']
# Зроблено тільки це, так як вже існує окрема колонка 'channelGrouping'

In [ ]:
print(sessions['channelGrouping'].unique())

### Device and platform categorizations

In [ ]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)